In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkDemo").getOrCreate()

df_test = spark.read.csv('data/test.csv', header=True, sep=';')
df_test.show(5)

+----+------+
| nom|prenom|
+----+------+
|test|  asdf|
|test|  qwer|
|test|  zxcv|
+----+------+



In [4]:
df_test.printSchema()

root
 |-- nom: string (nullable = true)
 |-- prenom: string (nullable = true)



Récupérer les informations de spark

In [6]:
print(spark.version)
print(spark.conf.get("spark.app.name"))
print(spark.sparkContext.master)

3.4.1
SparkDemo
local[*]


In [7]:
df_test.head(3)

[Row(nom='test', prenom='asdf'),
 Row(nom='test', prenom='qwer'),
 Row(nom='test', prenom='zxcv')]

In [8]:
df_test.select(['prenom']).show()

+------+
|prenom|
+------+
|  asdf|
|  qwer|
|  zxcv|
+------+



In [9]:
df_test.dtypes

[('nom', 'string'), ('prenom', 'string')]

In [10]:
df_test.describe().show()

+-------+----+------+
|summary| nom|prenom|
+-------+----+------+
|  count|   3|     3|
|   mean|null|  null|
| stddev|null|  null|
|    min|test|  asdf|
|    max|test|  zxcv|
+-------+----+------+

